In [ ]:
import sys
from pathlib import Path

_BASE_DIR = Path().resolve().parent.parent
print("(!) Make sure this dir is project directory: ", _BASE_DIR)
sys.path.append(str(_BASE_DIR))

from utils.settings import get_in_out_dirs

_LAB_NAME = "lab10"

INPUT_DIR, OUTPUT_DIR = get_in_out_dirs(base_name=_LAB_NAME)

DATA_DIR = INPUT_DIR / "data"
DATA_DIR.mkdir(parents=True, exist_ok=True)

(!) Make sure this dir is project directory:  /home/ars/Desktop/university/bsu-nn


# Лаб. 10

## Задание 1.  (из  тем  10.1–10.2):

Обучите рекуррентную нейронную сеть распознаванию тональности отзывов на тестовых данных открытого набора данных imdb.


In [2]:
import numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

2025-09-07 15:19:14.651385: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# -----------------------------
# Загружаем данные
# -----------------------------
NUM_WORDS = 5000   # используем топ-5000 слов
MAX_LEN = 200      # максимальная длина отзыва

(train_x, train_y), (test_x, test_y) = imdb.load_data(num_words=NUM_WORDS)

# Дополняем последовательности до одинаковой длины
train_x = pad_sequences(train_x, maxlen=MAX_LEN, padding='post')
test_x = pad_sequences(test_x, maxlen=MAX_LEN, padding='post')

print(f"Train shape: {train_x.shape}, Test shape: {test_x.shape}")

Train shape: (25000, 200), Test shape: (25000, 200)


In [ ]:
# -----------------------------
# Строим модель LSTM
# -----------------------------
embedding_dim = 32
lstm_units = 64

model = Sequential([
    Embedding(input_dim=NUM_WORDS, output_dim=embedding_dim),
    LSTM(lstm_units, dropout=0.2, recurrent_dropout=0.2),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

/home/ars/Desktop/university/bsu-nn/.venv/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2025-09-07 15:19:24.328349: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
MODEL_DIR = OUTPUT_DIR / "model"
MODEL_DIR.mkdir(parents=True, exist_ok=True)

OUT_MODEL_PATH = MODEL_DIR / "imdb_lstm_model.h5"

In [ ]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)

In [8]:
import os
from tensorflow.keras.models import load_model

# -----------------------------
# Обучение или загрузка
# -----------------------------

force_train = True

if not force_train and os.path.exists(OUT_MODEL_PATH):
    print("Модель найдена, загружаем вместо тренировки...")
    model = load_model(OUT_MODEL_PATH)

else:
    print("Модель не найдена, тренируем...")
    history = model.fit(
        train_x, train_y,
        epochs=1,
        batch_size=32,
        validation_split=0.2
    )
    print("Сохраняем модель...")
    model.save(OUT_MODEL_PATH)
    print(f"Модель сохранена в {OUT_MODEL_PATH}")

Модель не найдена, тренируем...


/home/ars/Desktop/university/bsu-nn/.venv/lib/python3.12/site-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


219/625 ━━━━━━━━━━━━━━━━━━━━ 16:12 2s/step - accuracy: 0.5548 - loss: 0.6887

KeyboardInterrupt: 

In [ ]:
# -----------------------------
# Оценка модели на тесте
# -----------------------------
loss, acc = model.evaluate(test_x, test_y)
print(f"Test loss: {loss:.4f}, Test accuracy: {acc:.4f}")

782/782 ━━━━━━━━━━━━━━━━━━━━ 10s 13ms/step - accuracy: 0.5296 - loss: 0.8406
Test loss: 0.8406, Test accuracy: 0.5296


## Задание 2.  (из  тем  10.1–10.2):

С использованием предварительно обученной нейронной сети определите тональность своего отзыва.

Прилагаю файл reviews retrieving - скрипт, позволяющий восстанавливать тексты отзывов с Intenet Movie DataBase.

В качестве входных данных нужно предоставить текст своего отзыва (можно написать прямо в окне "Ответ в виде текста").

В качестве результатов, пожалуйста, сдайте обученную рекуррентную нейронную сеть и результат распознавания ею Вашего отзыва.

In [10]:
# -----------------------------
# Распознать мой отзыв
# -----------------------------

my_review = "This movie was amazing! The plot was engaging and the characters were believable."

# Загружаем словарь IMDB
word_to_id = imdb.get_word_index()
word_to_id = {k:(v+3) for k,v in word_to_id.items()}
word_to_id["<PAD>"] = 0
word_to_id["<START>"] = 1
word_to_id["<UNK>"] = 2
word_to_id["<UNUSED>"] = 3

# Конвертируем отзыв в последовательность индексов
def text_to_sequence(text):
    import re
    sequence = []
    words = re.sub(r"[^\w\s]", "", text.lower()).split()
    for word in words:
        index = word_to_id.get(word, 2)  # <UNK>
        sequence.append(index)
    return sequence

my_seq = pad_sequences([text_to_sequence(my_review)], maxlen=MAX_LEN, padding='post')

# Предсказание
pred = model.predict(my_seq)[0][0]
print(f"Тональность вашего отзыва: {'Положительная' if pred>=0.5 else 'Отрицательная'} ({pred:.4f})")

/home/ars/Desktop/university/bsu-nn/.venv/lib/python3.12/site-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Тональность вашего отзыва: Отрицательная (0.4457)
